# Imports

In [49]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
import json

Creating our Spark Session and suppressing harmless warning messages to reduce clutter in the notebook

In [ ]:

spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("FATAL")

Reading our data from the data lake. The source is a JSON formatted multi-line file

In [51]:
df = spark.read.option("multiline", "true").json("../parsed_output/all_dns.json")
    

We select only the DNS specific/relevant fields from the dataframe

In [52]:
dns_data = df.select("timestamp","src_ip", "dst_ip","id", "opcode","qr","rcode","questions","answers")

We split the datasets into various categories. Valid queries and responses are those with a non-empty question or answer. Questions have ```qr = 0 ``` while answers have ``` qr == 1```. 

In [53]:
all_queries = dns_data.filter((dns_data["opcode"] == 0) & (dns_data["qr"] == 0))
all_responses = dns_data.filter((dns_data["opcode"] == 0) & (dns_data["qr"] == 1))
valid_queries = all_queries.filter(sf.size("questions") > 0).drop("answers")
valid_responses = all_responses.filter(sf.size("answers") > 0).drop("questions")
empty_questions = all_queries.filter(sf.size("questions") == 0)
empty_answers = all_responses.filter(sf.size("answers") == 0)

The summary dataframe below gives us an initial feel of the breakdown of our dataset. We will perform our analysis on specific datasets, and also correlate between datasets later for deeper insights.

In [54]:
summary_data =[{"Total DNS Records": dns_data.count(),
                "All Queries": all_queries.count(),
                "All Responses": all_responses.count(),
                "Valid (non-empty) Questions": valid_queries.count(),
                "Valid (non-empty) Answers": valid_responses.count()
                }]
summary = spark.createDataFrame(summary_data)

We now explode the questions arrays to extract the individual fields from the DNS request

In [55]:
valid_queries = valid_queries.withColumn("questions", sf.explode("questions"))
valid_queries = valid_queries.withColumns({
    "qname": valid_queries.questions.qname,
    "qtype": valid_queries.questions.qtype,
    "qlen": sf.length(valid_queries.questions.qname)}).drop("questions")

Similary, we explode the answers array to extract the individual answer fields

In [56]:
valid_responses = valid_responses.withColumn("answers", sf.explode("answers")).drop("questions")
valid_responses = valid_responses.withColumns({
    "rclass":valid_responses.answers.rclass,
    "rdata": valid_responses.answers.rdata,
    "rrname": valid_responses.answers.rrname,
    "rtype": valid_responses.answers.rtype,
    "ttl": valid_responses.answers.ttl
}).drop("answers", "opcode")


We rename the fields in the ```valid_responses``` dataframe to avoid conflicts with the ```valid_queries``` dataframe when we perform a join later.

In [57]:
valid_responses = valid_responses.withColumnsRenamed({
    "timestamp": "ts",
    "src_ip": "ns",
    "dst_ip": "client_ip",
    "id": "rid",
    "qr": "rqr",
    "rcode": "rrcode"
})

Now that we have prepared our dataset, we will persist to storage as ```.parquet``` format, for analysis.

In [58]:
valid_queries.write.parquet("../datasets/valid_queries.parquet", mode="overwrite")
valid_responses.write.parquet("../datasets/valid_responses.parquet", mode="overwrite")